In [92]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

In [93]:
#Inspired from Andrew Ng's course on Machine Learning
def logisticRegression():
    
    data = pd.read_csv("logReg.csv")
    
    X=data.iloc[:,:-1].values
    y=data.iloc[:,-1].values

    # Feature mapping 
    def featureMap(x_1,x_2,deg): # Generating All plausible Polynomial terms upto degree 'deg'

        res = np.ones(len(x_1)).reshape(len(x_1),1)
        for i in range(1,deg+1):
            for j in range(i+1):
                terms= (x_1**(i-j) * x_2**j).reshape(len(x_1),1)
                res= np.hstack((res,terms))
        return res

    X = featureMap(X[:,0], X[:,1],6)
    
    def sigmoid(k):
        z=(1 + np.exp(-k))
        return 1/z 

    def costFunctionReg(theta, X, y ,Lambda):

        #Regularization and Computes Gradient
    
        m=len(y)
        y=y[:,np.newaxis]
        pred = sigmoid(X @ theta) # @ - matrix multiplication 
        error = (-y * np.log(pred)) - ((1-y)*np.log(1-pred))
        cost = 1/m * sum(error)
        regCost= cost + Lambda/(2*m) * sum(theta**2)
        
        # compute gradient ( Gradient Descent )
        j_0= 1/m * (X.transpose() @ (pred - y))[0]
        j_1 = 1/m * (X.transpose() @ (pred - y))[1:] + (Lambda/m)* theta[1:]
        grad= np.vstack((j_0[:,np.newaxis],j_1)) # returns j_0 and j_1
    
        return regCost[0], grad
    
    initial_theta = np.zeros((X.shape[1], 1))

    # Set regularization parameter lambda to 1
    Lambda = 1

    #Compute and display initial cost and gradient for regularized logistic regression
    cost, grad=costFunctionReg(initial_theta, X, y, Lambda)
    
    
    # Genereating Noise
    def noisevector( scale, Length ):

        r1 = np.random.uniform(0, 1, Length)#standard normal distribution
        n1 = np.linalg.norm( r1, 1 )#get the norm of this random vector
        r2 = r1 / n1#the norm of r2 is 1
        normn = np.random.gamma( Length, scale, 1 )#Generate the norm of noise according to gamma distribution
        res = r2 * normn#get the result noise vector
        return res


    def gradientDescent(X,y,theta,alpha,num_iters,Lambda):

        # Generate theta (weights)
    
        m=len(y)
        J_history =[]
    
        for i in range(num_iters):
            cost, grad = costFunctionReg(theta,X,y,Lambda)
            theta = theta - (alpha * grad) 
            J_history.append(cost)
    
        return theta , J_history
    
    theta , J_history = gradientDescent(X,y,initial_theta,1,900,0.2)
    
    return theta

In [94]:
# Calculates distributed diff. private noise values for single weight using secret sharing and S trick

def calculate(n,weight_i): # n - number of parties/clients , weight_i - ith weight
    
    data = pd.read_csv("logReg.csv")  
    X=data.iloc[:,:-1].values
    
    
    foo=[]
    s = [[foo for i in range(n)] for j in range(n)]
    t=[]
    noiseS = [[t for i in range(n)] for j in range(n)]
    nv=[]
    noise = [[nv for i in range(n)] for j in range(n)]
    
    # Parties calculate shares and DP noise and send it to others
    for i in range(n):
    
        theta = logisticRegression()
        x=theta[weight_i] 
        Lambda=1
        epsilon=1 # diff private loss parameter
        scale = 2/( len(X) * Lambda * epsilon)
        
        w=[]
        r=[]     
    
        # generate random values for secret sharing
        for j in range(n-1):          
            r.append(random.randint(1,100))
           
        
        #generate random values for S trick
        
        for j in range(n):          
            s[i][j]=random.randint(1,1000)
           
        sum=0
        count=0
        #secret sharing
        for j in range(n):
            if (j==0):
                for k in range(n-1):
                    sum += r[k]
                w.append(x-sum)
    
            else:
                w.append(r[j-1])
              
            pickle.dump( w[j]+s[i][j], open( "weight"+str(i)+str(j)+".p", "wb" ) ) 
            
         
        #generate random values for S trick
        
        for j in range(n):          
            noiseS[i][j]=random.randint(1,1000)
            
                    
        #generate DP noise vector (Gamma - Gamma) 
        
        for j in range(n):
            a=np.random.gamma(1/n,scale, 1)
            b=np.random.gamma(1/n,scale, 1)
            noise[i][j]= b-a
            pickle.dump( noise[i][j]+noiseS[i][j], open( "noise"+str(i)+str(j)+".p", "wb" ) ) 


            
#----------------------------------------------------------------------------------------------------------------#

    #Parties calculate their share to be sent to server
    for i in range(n):
        
        sum=0      
        for j in range(n):        
            sum = sum + pickle.load( open( "weight"+str(j)+str(i)+".p", "rb" ) ) + pickle.load( open( "noise"+str(j)+str(i)+".p", "rb" ) )- noiseS[i][j]-s[i][j]
        
        pickle.dump( sum, open( "ro"+str(i)+".p", "wb" ) )  
        
#----------------------------------------------------------------------------------------------------------------#    
    #Server calculates weighted mean
    
    sum=0      
    for i in range(n):
        sum += pickle.load( open( "ro"+str(i)+".p", "rb" ) )
   
    print(sum/n)
            
        

In [95]:
calculate(2,1)
    
    

[1.51938677]
